In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
nodes=pd.read_csv('nodes_df.csv')
edges=pd.read_csv('edges_df.csv')

In [3]:
nodes=nodes.set_index('id')

In [5]:
guide_point_id='0P463'
guide_point_ll = [-71.087255, 42.364011]

In [7]:
guide_point_xy=nodes.loc[guide_point_id, ['x', 'y']]

In [9]:
guide_gdf=gpd.GeoDataFrame(data=[], geometry=gpd.points_from_xy(
    x=[guide_point_ll[0]], y=[guide_point_ll[1]]), crs='EPSG:4326')

In [11]:
utm_zone = int(np.floor((guide_point_ll[0] + 180) / 6) + 1)
print(utm_zone)
crs = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

19


In [12]:
guide_gdf_proj=guide_gdf.to_crs(crs)
guide_point_proj=guide_gdf_proj.iloc[0].geometry

### Reset the zero to the guide point and convert measurements to meters

In [15]:
nodes['x_ref']=(nodes['x']-guide_point_xy['x'])*0.3048
nodes['y_ref']=(nodes['y']-guide_point_xy['y'])*0.3048

In [16]:
nodes['x_proj']=nodes['x_ref']+guide_point_proj.x
nodes['y_proj']=nodes['y_ref']+guide_point_proj.y

In [17]:
nodes_proj_gdf=gpd.GeoDataFrame(data=nodes, geometry=gpd.points_from_xy(
    x=nodes['x_proj'], y=nodes['y_proj']), crs=crs)

In [18]:
nodes_wgs_gdf=nodes_proj_gdf.to_crs('EPSG:4326')

In [19]:
centroid=nodes_wgs_gdf.centroid
nodes_wgs_gdf['lon']=[c.x for c in centroid]
nodes_wgs_gdf['lat']=[c.y for c in centroid]

C:\Users\adminlocal\AppData\Local\Temp\ipykernel_3796\3753745346.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid=nodes_wgs_gdf.centroid


In [20]:
nodes_wgs_gdf[['lon', 'lat', 'z', 'type_of_entity', 'EntityHandle']].to_csv('nodes_wgs84.csv')